In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy 
import os
# import seaborn as sns
# import scipy.stats as stats
import unittest

## Battery Dataset
All battery expriments from snl

In [2]:
metadata = pd.read_csv('snl_metadata_cycle_500.csv')

In [3]:
data_dir = "Cycle Data"
files = os.listdir(data_dir)

In [4]:
df = pd.read_csv(data_dir+"\\"+files[0])
df_output = df.head(0)

In [5]:
cycle = [50.0, 100.0, 150.0, 200.0, 250.0, 300.0, 350.0, 400.0, 450.0, 500.0]

In [6]:
cell_id_2 =[]

i=0                
for file in files:
    df = pd.read_csv(data_dir+"\\"+files[i])
    
    for j in range(len(cycle)):
        df_cycles = df.loc[df['Cycle_Index'] == cycle[j]]
        df_output = df_output.append(df_cycles, ignore_index = True)
        
        df_output['Coulombic_Efficiency (%)'] = df_output['Discharge_Capacity (Ah)']/df_output['Charge_Capacity (Ah)']*100
        
        cell_id_2.append(file[:-15])
        df_output['cell_id_2'] = cell_id_2
        
        
    i+=1    
        

In [7]:
df_output = pd.concat([metadata, df_output],axis=1)

In [8]:
data_dir2 = "Time Data"
files2 = os.listdir(data_dir2)

In [9]:
df_output['Voltages (V)'] = None 
df_output['Discharge_Capacities (Ah)'] = None 

In [ ]:
voltages = []
discharge_capacities = []
i=0     
for file in files2:  
    df2 = pd.read_csv(data_dir2+"\\"+files2[i])
    
    for j in range(10):
        df_time = df2.loc[df2['Cycle_Index'] == df_output['Cycle'][j]]
        voltages.append(df_time['Voltage (V)'].values)
        discharge_capacities.append(df_time['Discharge_Capacity (Ah)'].values)
    i+=1    

In [ ]:
for i in range(len(df_output)):
    df_output['Voltages (V)'][i] = voltages[i]
    df_output['Discharge_Capacities (Ah)'][i] = discharge_capacities[i]

In [ ]:
df_battery = df_output

In [ ]:
for i in range(len(df_battery)):
    if df_battery['cell_id'][i] == df_battery['cell_id_2'][i]:
        pass
    else:
        print('not a match', df_battery['cell_id'][i],df_battery['cell_id_2'][i])

In [ ]:
df_battery = df_battery.drop('cell_id_2', 1)
df_battery = df_battery.drop('Cycle_Index', 1)

In [ ]:
df_battery

## Battery Dataset Filtered
Remove all repetative sets

In [ ]:
df_battery['rep'] = None 

In [ ]:
for i in range(len(df_battery)):
    if df_battery.cell_id.values[i][-1] == 'a':
        df_battery['rep'][i] = 0
    if df_battery.cell_id.values[i][-1] != 'a':
        df_battery['rep'][i] = 1

In [ ]:
df_battery_filtered = df_battery[df_battery['rep'] < 0.5]
df_battery_filtered = df_battery_filtered.drop('rep', 1)
df_battery_filtered = df_battery_filtered.reset_index(drop=True)

In [ ]:
df_battery_filtered.head(20)

## Battery Description
Takes an array created by user input components into GUI and returns a numerical array to input into machine learning models

In [ ]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
for i in range(len(df_battery_filtered)):
    if df_battery_filtered.at[i,'anode'] == 'graphite':
        df_battery_filtered.at[i,'anode'] = 0
    if df_battery_filtered.at[i,'cathode'] == 'LFP':
        df_battery_filtered.at[i,'cathode'] = 0
    if df_battery_filtered.at[i,'cathode'] == 'NCA':
        df_battery_filtered.at[i,'cathode'] = 1
    if df_battery_filtered.at[i,'cathode'] == 'NMC':
        df_battery_filtered.at[i,'cathode'] = 2
    if df_battery_filtered.at[i,'electrolyte'] == 'A123':
        df_battery_filtered.at[i,'electrolyte'] = 0
    if df_battery_filtered.at[i,'electrolyte'] == 'Pan':
        df_battery_filtered.at[i,'electrolyte'] = 1
    if df_battery_filtered.at[i,'electrolyte'] == 'LGC':
        df_battery_filtered.at[i,'electrolyte'] = 2

In [ ]:
df_battery_filtered.head(5)

In [ ]:
# cath = ['LFP']
# elec = ['A123']
# cy = 50
# temp = 15
# dcr = 1.0

In [ ]:
description = ['LFP', 'A123', 50, 15, 1.0] #What I expect to come out of GUI

In [ ]:
battery_anode = {
    0: 'Graphite'
}

battery_cathode = {
    0: 'LFP',
    1: 'NCA',
    2: 'NMC',
}

battery_electrolyte = {
    0: 'A123',
    1: 'Pan',
    2: 'LGC',
}

In [ ]:
desc = []
for key, value in battery_cathode.items():
    if value in list(battery_cathode.values())[0] == description[0]:
        desc.append(key)
for key, value in battery_electrolyte.items():
    if value in list(battery_electrolyte.values())[0] == description[1]:
        desc.append(key)
desc.append(description[2])
desc.append(description[3])
desc.append(description[4])
desc